## Application Flow

Before proceeding with the algorithm, let’s first discuss the lifecycle of any machine learning model. This diagram explains the creation of a Machine Learning model from scratch and then taking the same model further with hyperparameter tuning to increase its accuracy, deciding the deployment strategies for that model and once deployed setting up the logging and monitoring frameworks to generate reports and dashboards based on the client requirements. 
A typical lifecycle diagram for a machine learning model looks like:

<img src="MLApplicationFlow_bold.PNG" width= "300">

## Boosting
Boosting is an ensemble approach(meaning it involves several trees) that starts from a weaker decision and keeps on building the models such that the final prediction is the weighted sum of all the weaker decision-makers.
The weights are assigned based on the performance of an individual tree.

<img src= "boosting_basic.png" alt='boosting' style="width: 400px;">


Ensemble parameters are calculated in **stagewise way** which means that while calculating the subsequent weight, the learning from the previous tree is considered as well.


### Weak classifier - why tree?
First what is a weak classifier?
**Weak classifier** -  *slightly better* than random guessing.

Any algorithm could have been used as a base for the boosting technique, but the reason for choosing trees are:

#### Pro's
- computational scalability,
- handles missing values,
- robust to outliers,
- does not require feature scaling,
- can deal with irrelevant inputs,
- interpretable (if small),
- handles mixed predictors as well (quantitive and qualitative)

#### Con's
- inability to extract a linear combination of features
- high variance leading to a small computational power

And that’s where boosting comes into the picture. It minimises the variance by taking into consideration the results from various trees.


In every machine learning model, the training objective is a sum of a loss function $L$ and regularisation $\Omega$:

$$
obj = L + \Omega
$$

The loss function controls the predictive power of an algorithm and the regularisation term controls its simplicity.

There are several algorithms which use boosting. A few are discussed here.

### Ada Boost (Adaptive Boosting)
The steps to implement the Ada Boost algorithm using the decision trees are as follows:

**Algorithm**:

Assume that the number of training samples is denoted by $N$, and the number of iterations (created trees) is $M$. Notice that possible class outputs are $Y=\{-1,1\}$

1. Initialize the observation weights  $w_i=\frac{1}{N}$ where $i = 1,2, \dots, N$ for all the samples.
2. For $m=1$ to $M$:
    - fit a classifier $G_m(x)$ to the training data using weights $w_i$,
    - compute $err_m = \frac{\sum_{i=1}^{N} w_i I (y_i \neq G_m(x))}{\sum_{i=1}^{N}w_i}$,
    - compute $\alpha_m = \frac {1}{2} \log (\frac{(1-err_m)}{err_m})$. This is the contribution of that tree to the final result.
    - calculate the new weights using the formula:
    
    $w_i \leftarrow w_i \cdot \exp [\alpha_m \cdot I (y_i \neq G_m(x)]$, where $i = 1,2, \dots, N$
- Normalize the new sample  weights so that their sum is 1.
- Construct the next tree using the new weights


 3. At the end, compare the summation of results from all the trees and the final result is either the one with the highest sum(for regression) or it is the class which has the most weighted voted average(for classification).

       Output $G_m(x) = argmax [\sum_{m=1}^{M} \alpha_m G_m(x)]$ (Regression)

       Output $G_m(x) = sigm [\sum_{m=1}^{M} \alpha_m G_m(x)]$ (Classification)


**Example**

For understanding this algorithm, we'll use the following simple dataset for heart  patient prediction.

In [22]:
import pandas as pd
heart_data= pd.read_csv('heart_disease.csv')
heart_data

,Is Chest Pain Present,Are any arteries blocked,Weight of the person,Is Heart Patient
0,YES,YES,205,YES
1,NO,YES,180,YES
2,YES,NO,210,YES
3,YES,YES,167,YES
4,NO,YES,156,NO
5,NO,YES,125,NO
6,YES,NO,168,NO
7,YES,YES,172,NO


- There are a total of 8 rows in our dataset. Hence, we’ll initialize the sample weights($w=\frac {1}{N}$, where N is the total records.) as 1/8 in the beginning. And, at the beginning, all the samples are equally important.

<img src='sw1.PNG' width=”500”>

- We’ll consider the individual columns to create weak decision-makers as shown below and then try to figure out what are the correct and incorrect predictions based on that column.
- This images given below are called stumps.`Stumps are simply defined as the the tree having one parent node and two leaf node.`

<img src='cp1.PNG' width=”200”>

<img src='ba1.PNG' width=”200”>

<img src='bw1.PNG' width=”200”>

- We’ll now calculate the Gini index of the individual stumps using the formula

     G.I= $\sum (weight\_of\_the\_decision)*(1-(p^2+(1-p)^2))$

        G.I for chest pain tree= 0.47
        G.I for blocked arteries tree= 0.5
        G.I for body-weight tree= 0.2
        
        And, we select the tree with the lowest Gini Index. Here body-weight column will be the first decision-maker for our model.

- Now, we’ll calculate the contribution of this tree(stump) i.e body-weight to our final decision using the formula:

$$ total\_error = \frac {Total\_nr\_of\_incorrectly\_classified\_results}{Total\_nr\_of\_records}$$


$$ Contribution = ½ \left(log \left(\frac {(1-total\_error)}{(total\_error)}\right)\right)$$

- As this stump classified only one data incorrectly out of the 8, hence the total error is 1/8.

    Putting this into the above contribution formula  we get `Contribution= 0.97`
    
- We’ll now calculate the new weights using the formula:

1. Increase the sample weight for incorrectly classified datapoints

    New weight= $ old{\_weight}*e^{contribution}= {1/8 }* e^{0.97} = {0.33}$
    
1. Decrease the sample weight for incorrectly classified datapoints

   New weight= $ old{\_weight}*e^{-contribution} = {1/8 }* e^{-0.97} = {0.05}$

- Populate the new weights as shown below:

     <img src='nsw1.PNG' width=”300”>

- Normalize the sample weights: If we add all the new sample weights, we get 0.68. Hence, for normalization we divide all the sample weights by 0.68 and then create normalized sample weights as shown below: 

     <img src='normalized_wt.PNG' width=”200”>

       These new normalized weights will act as the sample weights for the next iteration.

- Then we create new trees which consider the dataset which was prepared using the new sample weights.
> `Explaination of the above sentence `
>> The normailized weights are between 0-1 and for the next tree we need a new dataset.For creation of new dataset we need this older data set.In the process of creating new dataset,first we select a random value between 0-1,by using this value we select the rows.
> How can we use this random value?
>> First we create range of the normalized weights(cumulative sum), like for the above dataset the range should be -
>> 0.07 - 0.14, 0.14 -0.21, 0.21 - 0.49, 0.49 - 0.56 and so on.
>> if we select a random value 0.28 and this value falls in the range 0.21-0.49 then the row corresponding  to the weight 0.49 is added to the new dataset.
>> Similarly if we select the random value is 0.08 and this value falls in the range 0.14-0.21 then the row corresponding to the weight 0.21 is added to the new dataset.This process goes on until the shape of new datset is equals to older one.

- The benefit of creating the new dataset using this method is the row which are incorrectly classified the results are selected more than once.These rows are learned by new stumps.
- In this way we have lots of stumps.

- Suppose, m trees(stumps) are classifying a person as a heart patient and n trees(stumps) are classifying a person as a healthy one, then the contribution of m and n trees are added separately and whichever has the higher value, the person is classified as that. 

_For example, if the contribution of m trees is 1.2 and the contribution of n trees is 0.5 then the final result will go in the favour of m trees and the person will be classified as a heart patient._


# Gradient Boosted Trees
*Gradient Boosted Trees* use decision trees as estimators. It can work with different loss functions (regression, classification, risk modelling etc.), evaluate it's gradient and approximates it with a simple tree (stage-wisely, that minimizes the overall error).

AdaBoost is a special case of Gradient Boosted Tree that uses exponential loss function.

**Gradient Boost Algorithm is used for both Classification and Regression**

# Note - Number of leaves used in the GD boost during used cases is between 8-32 but here we limit the tree with only 3 leafs.

#### Example

For understanding this algorithm we'll use the following simple dataset for weight prediction

In [1]:
import pandas as pd
weight_data= pd.read_csv('weights.csv')
weight_data

,Person Height(in metres),Person Favorite Colour,Person Gender,Person Weight (in Kg)
0,1.6,Blue,Male,88
1,1.6,Green,Female,76
2,1.5,Blue,Female,56
3,1.8,Red,Male,73
4,1.5,Green,Male,77
5,1.4,Blue,Female,57


# Gradient Boost For Regression
- For the first iteration,we predicted everyones weight is 71.2, which is the average of the target column(weight here) .

 Average=(88+76+56+73+77+57)/6=  71.2

- We consider this as the first prediction and then we’ll calculate the residual which is the difference between the predicted and the actual value as shown below:

  Like in the first row the residual is (88 - 71.2) = 16.8
<img src='pseudo_residuals.png' width='500'>


- Now we build a tree to fit these residuals as shown below:

<img src='residual_tree.PNG' width=”300”>

- We are only building the tree till a limited depth just for simplicity.
- As you can see, some leaves have more than one residuals. For those, we’ll calculate the average and the final tree will look like:

<img src='average_residual_tree.PNG' width=”300”>

- Now for prediction, we use the formula:-

                        New value= old value+learning rate * residual
If we consider the learning rate as 0.1, the result becomes.

                        New value=  71.2+0.1*16.8= 72.9 (for the first row).
-  Similarly the new predictions for all the rows is calculated.
    
- After we calculate new weight using the tree we created.We find new residuals which is the difference between the actual weights and the newly predicted weights.

- Then again fit the new residuals, to a new tree and predict the new weights using both the trees.

- `Note-` At every iteration we create a new tree.Each time when we predict the new weight we use all the tree we created before that. using this formula -

 
`New_weights = First Prediction(which is the average of the target column) + learning_rate * selected_residual_from_the_first_tree+ learning rate* selected_residual_from_the_second_tree + and so on for all the trees we created in the model.`

- By selected residual we mean that in the data set , each rows have some condition , when we pass that row in a tree , it predict some residual , which is the selected residual for that row.

- The above steps are repeated until there is no significant improvement in residuals.
- After that the final model is created.

**Predicting new data**

- When the new data comes the final result given by:-
<img src='new_data.PNG' width=”300”>

`Final Value= First Prediction  + learning rate * 1st residual+ learning rate* 2nd residual + and so on`



***

# Maths Behind Gradient Boost For Regression
- `Input`: Data $ \{(x_i,y_i)\}^{n}_{i=1}$    and differentiable `Loss function` $L(y_i,F(x))$.
---
**Pseudo Residuals**

   Here we use loss function $ \frac{1}{2} \left(Observed - Predicted \right)^2$  we are taking this loss function because the derivative w.r.t `Predictive` is  $ -\left(Observed - Predicted \right)$,  we can find it easily and we will end up calculating normal residuals.But when we use this $ \left(Observed - Predicted \right)^2$ , we are end up calculating $ -2\left(Observed - Predicted \right)$ this.and this is the `Pseudo Residuals`.
   
---
   
**Step 1: Initialize model with a constant value: $F_{0}(x) = argmin \sum^{n}_{i=1}L(y_i,\gamma)$**

- This is defined as finding the value of $\gamma$ or $predicted$ value which minimizes the sum of loss.

- Look for this dataset - 
<img src='data.PNG' width=”3” height = '5'>

- Here we use loss function $ \frac{1}{2} \left(Observed - Predicted \right)^2$ 

- The sum of loss functions are :
$$ \frac {1}{2}*\left(88 -Predicted \right)^2 + \frac {1}{2}*\left(76 -Predicted \right)^2 + \frac {1}{2}*\left(56 - Predicted \right)^ 2$$

- To find the minimum value of `Predicted` we need to find the sum of the derivatives w.r.t `Predictive` of all the loss functions and then equating to zero.

- So we get:-
> <img src='equation.PNG' width=”3” >
   $$ Predicted = \frac {88 + 76 + 56} {3} = 73.3$$

## Finally we get $F_0(x) = Predicted = 73.3 $ which is the initial prediction.

- <img src='Step2.PNG' width=”3” >

---

**Step_2:-**

*Explanation:-*
- The loop is between `m-M`.
-  `M` is total number of trees.
-  `m` is the current tree.
-  `i` is the current record.
-  `n` is the total number of records.
-  `r` is the Pseudo residuals.
-  `R` is the residuals in leaf node.
- `j` index of the residuals in leaf node.
- `J` total residual in single leaf node.
- $R_{jm}$ is the label on each leaf node.

- In `A` we are calculating the pseudo residuals from all the records  for the current tree.

- In `B` we are giving labels to the leaf node for the current tree. Suppose new data is passed and the leaf node is predicted by the tree, so the residual in that leaf node is considered for calculating the predicted value.

- In `C` we are calculating the actual residual of the leaf node because in leaf node there is more than one residuals.
  by the way the actual residual is the average of the the residuals in that node.
  > for example $\gamma_{(1,1)} = argmin \frac{1}{2}(56 - 73.3 + \gamma)^2$ and the differentiating it w.r.t  $\gamma$ and do this for all the records.
  
- In `D` we are calculating new weight using the model we trained.
**Given as**
- <img src='calculating.PNG' width=”3” height = '5'>

- This is for the first tree and we are doing good and this process goes on untill residuals are not started increasing.

# Gradient  Boost for Classification.
- Lots of steps are similar as the Gradient boost for classification.

**Step by Step explanation**
- We have a data set for training our gradient boost for classification model:-
<img src= 'Screenshot (81).png'>

- In Gradient Boost Regression we have continuous data in the target column, we easily find the average of them and it became the initial prediction for the model.But in the Gradient Boost Classification we have the discrete values in the target column.If we find the average of the target column but this average is not meaningful.So we need an initial prediction for this dataset to move further in building the model.

- We know that log(odds) from the logistic regression.When we use Gradient Boost Classification, the initial prediction for every records in the dataset is the log(odds).
- We need to find the log(odds) of the target columns.So the log(odds) for the people who loved troll_2 is given as:-

$$ log \left(odds\right) = log\left(\frac{4}{2}\right) = 0.7 $$

- After that we find the probability using the log(odds) value using the formula:-

$$ Probablity = \frac {e^{log(odds)}}{1 + e^{log(odds)}} $$

- For finding the probability we have to insert `log(odds)` values inside above formula we get:-
$$ Probability = \frac {e^{0.7}}{1 + e^{0.7}} = 0.7 $$
- <img src='Screenshot (82).png' width='800'>


- Suppose we are considering the threshold value for the model is `0.5`.
- <img src='Screenshot (83).png' width='800'>
---


- Here we consider the **Target** value `YES = 1` & `NO = 0`.
- We we have to calculate the residuals for every records, like for the first record -
$$ Residual = 1 - 0.7 = 0.3$$
 we calculate the residual for every rows for every rows.
- After calculating all the residual we built a tree for fitting thes residuals like we did for Gradient Boost for Regression.
-<img src='Screenshot (84).png' width='500'>
- After building the tree we see that some of the leafs are containg more than one residual values like Gradient Boost for Regression.But in Gradient Boost for Regression we simply find the average of the residuals in each leaf node which is the output of that leaf node.
- But here in case of Gradient Boost for Classification we can't find the residual by takig the average because the predictions are in terms of log(odds) and the leaf nodes of the residuals are derived from the Probability.
- <img src='Screenshot (86).png' width='500'>
- So we need to transform the residuals in the leaf node are in terms of log(odds). So for Transforming we need the formula which is given as:-

$$ New\_Residual\_for\_leaf = \frac {\sum_i(Residual_i)}{\sum_i[Previous\_Probability * (1 - Previous\_Probability)]} $$
 for derivation of the above formula Watch [Derivation](https://www.youtube.com/watch?v=StWY5QWMXCw&t=6s)
 
- Using the above formula we calculate the value of each leaf node in terms of log(odds).
- Calculating the value of the leaf node which is having only one residual value- 
-<img src='Screenshot (87).png' width='500'>
-<img src='Screenshot (89).png' width='500'>
-<img src='Screenshot (90).png' width='500'>
-<img src='Screenshot (91).png' width='500'>
-<img src='Screenshot (92).png' width='500'>

- Calculating the value of the leaf node which is having more than one residual value- 
-<img src='Screenshot (93).png' width='500'>
-<img src='Screenshot (94).png' width='500'>
-<img src='Screenshot (95).png' width='500'>
-<img src='Screenshot (96).png' width='500'>

- Similarly we calculate for all the three leafs and we get:-
-<img src='Screenshot (97).png' width='500'>

- After this step we need to update our prediction on the basis of this first tree:-
-<img src='Screenshot (98).png' width='500'>
-<img src='Screenshot (99).png' width='500'>
- So for the first person in the dataset the new predictions is:-
-<img src='Screenshot (100).png' width='500'>
-<img src='Screenshot (101).png' width='500'>
-<img src='Screenshot (102).png' width='500'>
 because the probability is greater than 0.5.
 
 - Similarly we calculate the probability for all the rows in the dataset and store it into new columns `Predicted Prob`. And also these probability is used as previous probability when find the values of the residuals in the leaf node in terms of log(odds). 
-<img src='Screenshot (103).png' width='500'>-

- After calculating the probability we need to calculate the residual for each rows using these probability.
- Calculation of residuals is given as - 
-<img src='Screenshot (104).png' width='500'>
-<img src='Screenshot (105).png' width='500'>
-<img src='Screenshot (106).png' width='500'>

- Similary we calculte for all the rows:-

-<img src='Screenshot (107).png' width='500'>
-<img src='Screenshot (108).png' width='500'>
-<img src='Screenshot (109).png' width='500'>
-<img src='Screenshot (110).png' width='500'>

- After calculating all the residual we build a new tree for fitting the new residuals.

-<img src='Screenshot (111).png' width='500'>
-<img src='Screenshot (112).png' width='500'>

- Calculating the value for each leaf node:-
-<img src='Screenshot (113).png' width='500'>
-<img src='Screenshot (114).png' width='500'>
-<img src='Screenshot (115).png' width='500'>
-<img src='Screenshot (116).png' width='500'>
-<img src='Screenshot (117).png' width='500'>
-<img src='Screenshot (118).png' width='500'>
-<img src='Screenshot (121).png' width='500'>
-<img src='Screenshot (122).png' width='500'>
-<img src='Screenshot (123).png' width='500'>

- Combining everything we done earlier and update the weights - 

-<img src='Screenshot (124).png' width='500'>
-<img src='Screenshot (125).png' width='500'>
-<img src='Screenshot (126).png' width='500'>
-<img src='Screenshot (127).png' width='500'>
-<img src='Screenshot (128).png' width='500'>
-<img src='Screenshot (129).png' width='500'>
-<img src='Screenshot (130).png' width='500'>
-<img src='Screenshot (131).png' width='500'>

---
- Suppose we made only two trees in the for our model then:-
-<img src='Screenshot (132).png' width='500'>
-<img src='Screenshot (133).png' width='500'>
-<img src='Screenshot (134).png' width='500'>
-<img src='Screenshot (135).png' width='500'>
-<img src='Screenshot (136).png' width='500'>
-<img src='Screenshot (137).png' width='500'>
-<img src='Screenshot (138).png' width='500'>
- Here predicted-
-<img src='Screenshot (139).png' width='500'>
